<a href="https://colab.research.google.com/github/fahriyegrl/experiment_track_pretrained_models/blob/main/experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms, datasets

try:
  from torchinfo import summary
except:
  !pip install -q torchinfo
  from torchinfo import summary

try:
  from going_modular.going_modular import data_setup, engine
except:
  print("[INFO] downloading going_modular from GitHub.")
  !git clone https://github.com/fahriyegrl/going_modular_pytorch
  !mv going_modular_pytorch/going_modular .
  !rm -rf going_modular_pytorch
  from going_modular.going_modular import data_setup,engine


device = "cuda" if torch.cuda.is_available() else "cpu"
device

[INFO] downloading going_modular from GitHub.
Cloning into 'going_modular_pytorch'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 8), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 13.10 KiB | 6.55 MiB/s, done.
Resolving deltas: 100% (8/8), done.


'cpu'

In [ ]:
## getting data

import os
import requests
import zipfile
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "pizza_sushi_steak"

if image_path.is_dir():
  print (f"{image_path} exists")
else:
  print(f"creating {image_path}")
  image_path.mkdir(parents=True, exist_ok=True)

  with open(data_path / "pizza_sushi_steak.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("downloading pizza_sushi_steak data...")
    f.write(request.content)

  with zipfile.ZipFile(data_path / "pizza_sushi_steak.zip", "r") as zip:
    zip.extractall(image_path)

train_dir = image_path / "train"
test_dir = image_path / "test"



creating data/pizza_sushi_steak
downloading pizza_sushi_steak data...


In [ ]:
from torch.utils.tensorboard import SummaryWriter
def create_writer(experiment_name: str,
                  model_name: str,
                  extra:str=None):
  from datetime import datetime
  import os

  timestamp = datetime.now().strftime("%Y-%m-%d")
  if extra:
    log_dir=os.path.join("runs", timestamp, experiment_name, model_name, extra)
  else:
    log_dir=os.path.join("runs", timestamp, experiment_name, model_name)
  print(f" Created SummaryWriter, saving to: {log_dir}")

  return SummaryWriter(log_dir=log_dir)


In [ ]:
writer = create_writer(experiment_name="experiment_test",
                       model_name="model_test",
                       extra="adding_extra")

 Creaated SummaryWriter, saving to: runs/2024-07-07/experiment_test/model_test/adding_extra


In [ ]:
len(test_dataloader)

3

In [ ]:
from typing import Dict, List
from tqdm.auto import tqdm
from going_modular.going_modular.engine import train_step, test_step

def train(model:torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List]:

    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc":[]
               }
    for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                         dataloader=train_dataloader,
                                         loss_fn=loss_fn,
                                         optimizer=optimizer,
                                         device=device)
      test_loss, test_acc = test_step(model=model,
                                         dataloader=test_dataloader,
                                         loss_fn=loss_fn,
                                         device=device)

      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss} |"
          f"train_acc: {train_acc} |"
          f"test_loss: {test_loss} |"
          f"test_loss: {test_acc} "
      )
      if writer:
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict= {"train_loss": train_loss,
                                             "test_loss": test_loss},
                           global_step=epoch)
        writer.add_scalars(main_tag="Accurancy",
                           tag_scalar_dict= {"train_acc": train_acc,
                                             "test_acc": test_acc},
                           global_step=epoch)
        writer.close()
      else:
        pass
    return results



In [ ]:

import os
import zipfile
from pathlib import Path
import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool= True):
  data_path = Path("data/")
  image_path = data_path / destination

  if image_path.is_dir():
    print(f"{image_path} exists")
  else:
    print(f"creating {image_path}")
    image_path.mkdir(parents=True, exist_ok=True)
    target_file = Path(source).name
    with open (data_path / target_file, "wb") as f:
      request = requests.get(source)
      f.write(request.content)

    with zipfile.ZipFile(data_path / target_file, "r") as zip:
      zip.extractall(image_path)

    if remove_source:
      os.remove(data_path / target_file)
  return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")

image_path



creating data/pizza_steak_sushi


PosixPath('data/pizza_steak_sushi')

In [ ]:
## Getting data with 2 options

data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")





data/pizza_steak_sushi exists
creating data/pizza_steak_sushi_20_percent


In [ ]:
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

test_dir = data_10_percent_path / "test"

train_dir_10_percent


PosixPath('data/pizza_steak_sushi/train')

In [ ]:
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize])



In [ ]:
BATCH_SIZE = 32

train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=transform,
                                                                                          batch_size=BATCH_SIZE)

train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=transform,
                                                                                          batch_size=BATCH_SIZE)



8

In [ ]:
## Pick a model from torchvision.models : efficientnet_v2_s

import torchvision
from torchvision import models

efficientnet_v2_s_weights = models.EfficientNet_V2_S_Weights.DEFAULT
efficientnet_v2_s = models.efficientnet_v2_s(weights=efficientnet_v2_s_weights)
# efficientnet_v2_s

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 149MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [ ]:
def create_effnetb2(out_features=len(class_names)):
  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  model = torchvision.models.efficientnet_b2(weights=weights).to(device)
  dropout = 0.3
  in_features=1408

  for param in model.parameters():
    param.requires_grad = False

  torch.manual_seed(42)

  model.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408,
                out_features=out_features)).to(device)

  model.name = "effnetb2"

  return model

def create_effnetv2_s(out_features=len(class_names)):
  weights = torchvision.models.EfficientNet_V2_S_Weights.DEFAULT
  model = torchvision.models.efficientnet_v2_s(weights=weights).to(device)
  dropout = 0.2
  in_features=1280

  for param in model.parameters():
    param.requires_grad = False

  torch.manual_seed(42)

  model.classifier = nn.Sequential(
      nn.Dropout(p=0.2, inplace=True),
      nn.Linear(in_features=in_features,
                out_features=out_features)).to(device)

  model.name = "effnetv2_s"

  return model





  ##(classifier): Sequential(
   ## (0): Dropout(p=0.2, inplace=True)
   ## (1): Linear(in_features=1280, out_features=1000, bias=True)

In [ ]:
model_effnetb2 = create_effnetb2()
model_effnetv2_s = create_effnetv2_s()

In [ ]:
from torchinfo import summary

summary(model=model_effnetv2_s,
        input_size =(1,3,224,224),
        col_names=["input_size","output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 24, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 24, 112, 112]    (648)                False
│    │    └─BatchNorm2d (1)                                  [1, 24, 112, 112]    [1, 24, 112, 112]    (48)                 False
│    │    └─SiLU (2)                                         [1, 24, 112, 112]    [1, 24, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 24, 112, 112]    [1, 2

In [ ]:
num_epochs = [5,10]
models = ["effnetb2", "effnetv2_s"]

train_dataloaders = {"data_10_percent": train_dataloader_10_percent,
                    "data_20_percent": train_dataloader_20_percent}

In [ ]:
from going_modular.going_modular.utils import save_model

torch.manual_seed(42)
experiment_number = 0

for dataloader_name, train_dataloader in train_dataloaders.items():
  for epochs in num_epochs:
    for model_name in models:
      experiment_number += 1
      print(f"Experiment number: {experiment_number}")
      print(f"Model: {model_name}")
      print(f"Dataloader: {dataloader_name}")
      print(f"Epochs: {epochs}")

      if model_name == "effnetb2":
        model = create_effnetb2()
      else:
        model= create_effnetv2_s()

      loss_fn = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

      train(model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            optimizer=optimizer,
            loss_fn=loss_fn,
            epochs=epochs,
            device=device,
            writer=create_writer(experiment_name=dataloader_name,
                                model_name=model_name,
                                extra=f"{epochs}_epochs"))

      save_filepath = f"{model_name}_{dataloader_name}_{epochs}_epochs.pth"
      save_model(model=model,
                 target_dir="models",
                 model_name=save_filepath)




Experiment number: 1
Model: effnetb2
Dataloader: data_10_percent
Epochs: 5
 Creaated SummaryWriter, saving to: runs/2024-07-08/data_10_percent/effnetb2/5_epochs


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0892456769943237 |train_acc: 0.3359375 |test_loss: 0.9344133734703064 |test_loss: 0.7102272727272728 
Epoch: 2 | train_loss: 0.8967178836464882 |train_acc: 0.66796875 |test_loss: 0.8437613248825073 |test_loss: 0.7935606060606061 
Epoch: 3 | train_loss: 0.8505054041743279 |train_acc: 0.6796875 |test_loss: 0.7501824895540873 |test_loss: 0.8854166666666666 
Epoch: 4 | train_loss: 0.7134237810969353 |train_acc: 0.75 |test_loss: 0.7178870240847269 |test_loss: 0.8560606060606061 
Epoch: 5 | train_loss: 0.6579812839627266 |train_acc: 0.80078125 |test_loss: 0.6829179326693217 |test_loss: 0.8465909090909092 
[INFO] Saving model to: models/effnetb2_data_10_percent_5_epochs.pth
Experiment number: 2
Model: effnetv2_s
Dataloader: data_10_percent
Epochs: 5
 Creaated SummaryWriter, saving to: runs/2024-07-08/data_10_percent/effnetv2_s/5_epochs


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0740615725517273 |train_acc: 0.46484375 |test_loss: 0.9162604014078776 |test_loss: 0.6912878787878788 
Epoch: 2 | train_loss: 0.9237552061676979 |train_acc: 0.64453125 |test_loss: 0.8222423593203226 |test_loss: 0.7632575757575758 
Epoch: 3 | train_loss: 0.798132911324501 |train_acc: 0.84375 |test_loss: 0.723977247873942 |test_loss: 0.8248106060606061 
Epoch: 4 | train_loss: 0.7359224855899811 |train_acc: 0.7421875 |test_loss: 0.6472240090370178 |test_loss: 0.8465909090909092 
Epoch: 5 | train_loss: 0.6897624433040619 |train_acc: 0.74609375 |test_loss: 0.5896820425987244 |test_loss: 0.8967803030303031 
[INFO] Saving model to: models/effnetv2_s_data_10_percent_5_epochs.pth
Experiment number: 3
Model: effnetb2
Dataloader: data_10_percent
Epochs: 10
 Creaated SummaryWriter, saving to: runs/2024-07-08/data_10_percent/effnetb2/10_epochs


...

Epoch: 1 | train_loss: 0.9926369547843933 |train_acc: 0.475 |test_loss: 0.7386114796002706 |test_loss: 0.8768939393939394 
Epoch: 2 | train_loss: 0.7570270697275797 |train_acc: 0.78125 |test_loss: 0.599230686823527 |test_loss: 0.8551136363636364 
Epoch: 3 | train_loss: 0.6305111606915792 |train_acc: 0.7875 |test_loss: 0.5305979947249094 |test_loss: 0.8456439393939394 
Epoch: 4 | train_loss: 0.6012333651383718 |train_acc: 0.8 |test_loss: 0.4771365423997243 |test_loss: 0.8967803030303031 
Epoch: 5 | train_loss: 0.5588782866795857 |train_acc: 0.8104166666666667 |test_loss: 0.45867762962977093 |test_loss: 0.8570075757575758 
[INFO] Saving model to: models/effnetv2_s_data_20_percent_5_epochs.pth
Experiment number: 7
Model: effnetb2
Dataloader: data_20_percent
Epochs: 10
 Creaated SummaryWriter, saving to: runs/2024-07-08/data_20_percent/effnetb2/10_epochs


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9892369230588277 |train_acc: 0.5291666666666667 |test_loss: 0.7823601563771566 |test_loss: 0.8446969696969697 
Epoch: 2 | train_loss: 0.726783295472463 |train_acc: 0.79375 |test_loss: 0.6626894076665243 |test_loss: 0.8873106060606061 
Epoch: 3 | train_loss: 0.6023216545581818 |train_acc: 0.8520833333333333 |test_loss: 0.5854155421257019 |test_loss: 0.9280303030303031 
Epoch: 4 | train_loss: 0.5311014254887899 |train_acc: 0.85625 |test_loss: 0.5396908223628998 |test_loss: 0.9071969696969697 
Epoch: 5 | train_loss: 0.5126108527183533 |train_acc: 0.8416666666666667 |test_loss: 0.45916470885276794 |test_loss: 0.9375 
Epoch: 6 | train_loss: 0.44320916930834453 |train_acc: 0.8916666666666667 |test_loss: 0.39814747373263043 |test_loss: 0.9479166666666666 
Epoch: 7 | train_loss: 0.39537834127744037 |train_acc: 0.91875 |test_loss: 0.40761133035024005 |test_loss: 0.9289772727272728 
Epoch: 8 | train_loss: 0.34670491913954415 |train_acc: 0.9270833333333334 |test_loss: 0.4

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9926369547843933 |train_acc: 0.475 |test_loss: 0.7386114796002706 |test_loss: 0.8768939393939394 
Epoch: 2 | train_loss: 0.7570270697275797 |train_acc: 0.78125 |test_loss: 0.599230686823527 |test_loss: 0.8551136363636364 
Epoch: 3 | train_loss: 0.6305111606915792 |train_acc: 0.7875 |test_loss: 0.5305979947249094 |test_loss: 0.8456439393939394 
Epoch: 4 | train_loss: 0.6012333651383718 |train_acc: 0.8 |test_loss: 0.4771365423997243 |test_loss: 0.8967803030303031 
Epoch: 5 | train_loss: 0.5588782866795857 |train_acc: 0.8104166666666667 |test_loss: 0.45867762962977093 |test_loss: 0.8570075757575758 
Epoch: 6 | train_loss: 0.47895973920822144 |train_acc: 0.85625 |test_loss: 0.4144737621148427 |test_loss: 0.8967803030303031 
Epoch: 7 | train_loss: 0.44333017269770303 |train_acc: 0.8604166666666667 |test_loss: 0.38914228479067486 |test_loss: 0.9166666666666666 
Epoch: 8 | train_loss: 0.4340198278427124 |train_acc: 0.8479166666666667 |test_loss: 0.4155457615852356 |te